# Checkpoint 2 Notebook

__TODO__:
- fill this cell in with info, results, insights from our EDA 
- preprocess: create preprocessing pipeline, store all code into script, then run script to preprocess all data and store into pickle
- analysis: need "3 hypothesis tests" to validate hypotheses